This notebook can be run after running 'nvi_2024_processor.py' with the cdo_aggregate flag set to 'True.' That will output the expected cdo tall file. In the future we'll incorporate this code more directly into the transform_data function so the CDOs will be aggregated along with everything else.

In [ ]:
from pathlib import Path
import pandas as pd
from sqlalchemy import text
from nvi_etl import make_engine_for
from nvi_etl.geo_reference import pull_cdo_boundaries

cdo_boundaries = pull_cdo_boundaries()
db_engine = make_engine_for("nvi_test")

indicator = pd.read_sql(
    text(
        """
        SELECT *
        FROM indicator;
        """
    ), db_engine
)

survey_question = pd.read_sql(
    text(
        """
        SELECT *
        FROM survey_question;
        """
    ), db_engine
)

survey_question_option = pd.read_sql(
    text(
        """
        SELECT *
        FROM survey_question_option;
        """
    ), db_engine
)


long = pd.read_csv(Path.cwd() / "output" / "primary_survey_tall_cdo_2024.csv")
cdos = long[~long["location_id"].str.isnumeric()]

In [ ]:
cdo_groups = (
    cdos
    .merge(indicator, left_on="indicator_id", right_on="id")
    .rename(columns={"name": "Indicator Name"})
    .merge(survey_question, left_on="survey_question_id", right_on="id", how="left")
    .merge(survey_question_option, left_on="survey_question_option_id", right_on="id", how="left")
    .rename(columns={
        "name": "Question Option",
        "question_text": "Question Text",
        "location_id": "CDO Name",
        "percentage": "Percentage",
        "count": "Count",
        "universe": "Total Respondents",
    })[[
        "CDO Name",
        "Indicator Name",
        "Question Text",
        "Question Option",
        "Percentage",
        "Count",
        "Total Respondents"
    ]]
    .groupby("CDO Name")
)

In [ ]:
for label, group in cdo_groups:
   SAVE_DIR = Path("P:\\2025_Projects\\NVI25\\Development\\Workspace\\CDO Data Cuts")
   cleaned_label = label.replace(":", "").replace("/", "-")

   group.to_csv(SAVE_DIR / f"{cleaned_label} NVI Cut 20251022.csv")
